In [1]:
import pandas as pd
import csv
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/la_airbnb/{}.csv'

listings_file = file_url.format('listings_details')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [3]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

sf_file = file_url.format('sf_airbnb/listings_details')
ok_file = file_url.format('ok_airbnb/listings_details')
sc_file = file_url.format('sc_airbnb/listings_details')
scz_file = file_url.format('scz_airbnb/listings_details')
pg_file = file_url.format('pg_airbnb/listings_details')
sd_file = file_url.format('sd_airbnb/listings_details')
la_file = file_url.format('la_airbnb/listings_details')

sea_file = file_url.format('sea_airbnb/listings_details')

In [4]:
listings_sf = pd.read_csv(sf_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_ok = pd.read_csv(ok_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_sc = pd.read_csv(sc_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_scz = pd.read_csv(scz_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_sd = pd.read_csv(sd_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_la = pd.read_csv(la_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])

listings_sea = pd.read_csv(sea_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])

In [5]:
print (listings_sf.shape, listings_ok.shape, listings_sc.shape, listings_scz.shape, listings_sea.shape,listings_la.shape)

(6633, 96) (2898, 96) (5668, 96) (1570, 96) (3818, 92) (42337, 96)


In [6]:
frames = [listings_la, listings_sea]

In [7]:
listings = pd.concat(frames)

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
listings.columns.values

array(['access', 'accommodates', 'amenities', 'availability_30',
       'availability_365', 'availability_60', 'availability_90',
       'bathrooms', 'bed_type', 'bedrooms', 'beds',
       'calculated_host_listings_count', 'calendar_last_scraped',
       'calendar_updated', 'cancellation_policy', 'city', 'cleaning_fee',
       'country', 'country_code', 'description', 'experiences_offered',
       'extra_people', 'first_review', 'guests_included',
       'has_availability', 'host_about', 'host_acceptance_rate',
       'host_has_profile_pic', 'host_id', 'host_identity_verified',
       'host_is_superhost', 'host_listings_count', 'host_location',
       'host_name', 'host_neighbourhood', 'host_picture_url',
       'host_response_rate', 'host_response_time', 'host_since',
       'host_thumbnail_url', 'host_total_listings_count', 'host_url',
       'host_verifications', 'house_rules', 'id', 'instant_bookable',
       'interaction', 'is_business_travel_ready', 'is_location_exact',
       'j

In [9]:
df = listings[["host_response_rate", "host_acceptance_rate", "host_is_superhost",
               "host_listings_count", "zipcode", "property_type","room_type", "accommodates", "bathrooms", "bedrooms", 
               "beds", "price", "number_of_reviews", "review_scores_rating", "cancellation_policy", 
               "reviews_per_month"]]

In [10]:
df.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,cancellation_policy,reviews_per_month
0,NaN,NaN,f,1.0,90230,Condominium,Entire home/apt,6,2.0,2.0,3.0,$122.00,2,80.0,strict_14_with_grace_period,0.02
1,100%,NaN,f,1.0,91505,House,Entire home/apt,6,1.0,3.0,3.0,$168.00,1,80.0,flexible,0.04
2,0%,NaN,f,1.0,90292,Apartment,Shared room,1,NaN,1.0,1.0,$85.00,0,NaN,moderate,NaN
3,100%,NaN,f,1.0,90046,Apartment,Private room,1,1.5,1.0,1.0,$79.00,11,96.0,strict_14_with_grace_period,0.22
4,100%,NaN,f,2.0,90405,Apartment,Private room,1,1.0,1.0,1.0,$140.00,17,96.0,strict_14_with_grace_period,0.19


In [11]:
# % of NaN values
(len(df)-df.count())/len(df)*100

host_response_rate      17.389232
host_acceptance_rate    93.402665
host_is_superhost        0.010833
host_listings_count      0.010833
zipcode                  1.126639
property_type            0.002167
room_type                0.000000
accommodates             0.000000
bathrooms                0.112664
bedrooms                 0.056332
beds                     0.084498
price                    0.000000
number_of_reviews        0.000000
review_scores_rating    22.129780
cancellation_policy      0.000000
reviews_per_month       20.704149
dtype: float64

In [12]:
df = df.drop(['host_acceptance_rate'], axis=1)

In [13]:
# drop NaN rows
df2=df.dropna(axis=0)

In [14]:
# % of rows droppped
(len(df)-len(df2))/len(df)*100

31.561044307225654

In [15]:
df2.shape

(31588, 15)

In [16]:
pd.options.mode.chained_assignment = None  # default='warn'

df2['host_response_rate'] = df2['host_response_rate'].astype(str)
df2['price'] = df2['price'].astype(str)

In [17]:
# clean data
pd.options.mode.chained_assignment = None  # default='warn'

df2['price'] = df2['price'].str.replace("[$, ]", "").astype("float")
df2['host_response_rate'] = df2['host_response_rate'].str.replace("%", "").astype("float")

In [18]:
df2['superhost']=np.where(df2['host_is_superhost']=='t',1,0)
del df2['host_is_superhost']

In [19]:
# select non-numeric variables and create dummies
non_num_vars = df2.select_dtypes(include=['object']).columns
df2[non_num_vars].head()

,zipcode,property_type,room_type,cancellation_policy
1,91505,House,Entire home/apt,flexible
3,90046,Apartment,Private room,strict_14_with_grace_period
4,90405,Apartment,Private room,strict_14_with_grace_period
6,90046,House,Entire home/apt,strict_14_with_grace_period
7,91604,House,Entire home/apt,strict_14_with_grace_period


In [20]:
dummy_vars = pd.get_dummies(df2[non_num_vars])

In [21]:
dummy_vars.head()

,zipcode_90001,zipcode_90002,zipcode_90003,zipcode_90004,zipcode_90005,zipcode_90006,zipcode_90007,zipcode_90008,zipcode_90010,zipcode_90011,...,property_type_Yurt,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [22]:
# drop non-numeric variables from df2 and add the dummies
df3=df2.drop(non_num_vars,axis=1)
df3 = pd.merge(df3,dummy_vars, left_index=True, right_index=True)
df3.head()

,host_response_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month,...,property_type_Yurt,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
0,96.0,3.0,4,1.0,1.0,1.0,85.0,207,95.0,4.07,...,0,1,0,0,0,1,0,0,0,0
1,100.0,1.0,6,1.0,3.0,3.0,168.0,1,80.0,0.04,...,0,1,0,0,1,0,0,0,0,0
1,100.0,1.0,6,1.0,3.0,3.0,168.0,1,80.0,0.04,...,0,1,0,0,0,0,1,0,0,0
1,98.0,6.0,4,1.0,1.0,1.0,150.0,43,96.0,1.48,...,0,1,0,0,1,0,0,0,0,0
1,98.0,6.0,4,1.0,1.0,1.0,150.0,43,96.0,1.48,...,0,1,0,0,0,0,1,0,0,0


In [23]:
df4 = pd.get_dummies(df2, prefix=non_num_vars, columns=non_num_vars)
df4.shape

(31588, 391)

In [24]:
# split into test and training data
np.random.seed(1)
indices = np.random.permutation(len(df4))
train_size = int(round(0.8*len(df4)))
test_size = len(df4)-train_size

y = df4['price']
x = df4.drop('price', axis =1)

x.train = x.iloc[indices[0:train_size]]
y.train = y.iloc[indices[0:train_size]]
x.test = x.iloc[indices[train_size+1:]]
y.test = y.iloc[indices[train_size+1:]]

x2 = x.train.as_matrix()
y2 = y.train.as_matrix()

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [25]:
import statsmodels.api as sm
olsmod = sm.OLS(y2,x2)
olsres = olsmod.fit()
print(olsres.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.409
Method:                 Least Squares   F-statistic:                     46.60
Date:                Sat, 15 Sep 2018   Prob (F-statistic):               0.00
Time:                        22:03:01   Log-Likelihood:            -1.6988e+05
No. Observations:               25270   AIC:                         3.405e+05
Df Residuals:                   24885   BIC:                         3.437e+05
Df Model:                         384                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.6182      0.114     -5.411      0.0

In [26]:
# different method
from sklearn import linear_model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=1)

In [27]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=200, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 4700.843, test: 30179.494
R^2 train: 0.929, test: 0.554
